In [30]:
import time
import ccxt
import numpy as np
import os

class GridTradingBot:

    def __init__(self, mode='paper', symbol='BTC/USDT', exchange_id='binance'):
        """
        Initialize the grid trading bot
        
        Parameters:
        - mode: 'paper', 'testnet', or 'live'
        - symbol: Trading pair
        - exchange_id: Exchange to use (binance, bybit, etc.)
        """
        self.mode = mode
        self.symbol = symbol
        self.base_currency = symbol.split('/')[0]
        self.quote_currency = symbol.split('/')[1]
        self.exchange_id = exchange_id
        
        # Trading parameters
        self.grid_levels = 10  # Number of grid levels
        self.grid_spacing = 500  # Price difference between grid levels (for arithmetic)
        self.ratio = 1.936204  # Geometric ratio
        self.grid_type = 'geometric'  # 'arithmetic' or 'geometric'
        self.order_size = 0.001  # Base order size in BTC
        self.leverage = 1  # Leverage multiplier
        self.initial_price = None  # Will be set when connecting to market
        
        # Exchange connection
        self.exchange = None
        self.connect_exchange()
        
        # Trading state
        self.grid_prices = []
        self.active_orders = {}
        self.last_trade = None
        self.last_trade_price = None
        self.completed_trades = 0
        self.total_profit = 0
        self.trades_history = []
        
        # Fees (Binance futures default)
        self.fees = {'maker': 0.0002, 'taker': 0.0005}  # 0.02%, 0.05%
        
        # Paper trading specific
        self.paper_balance = 10000  # Starting with 10000 USDT
        self.paper_positions = {}
        
        # Initialize grid
        self.update_market_price()
        self.generate_grid_prices()
        
        print(f"Grid Trading Bot initialized in {mode} mode")
        print(f"Symbol: {symbol}, Exchange: {exchange_id}")
        print(f"Initial price: {self.initial_price}")
        print(f"Grid type: {self.grid_type}")
        print(f"Grid levels: {self.grid_levels}")
        print(f"Leverage: {self.leverage}x")
    
    def connect_exchange(self):
        """Connect to exchange based on mode"""
        try:
            if self.mode == 'paper':
                # For paper trading, we still connect to exchange for price data
                self.exchange = ccxt.binance()
                print("Connected to paper trading (simulated).")
            elif self.mode == 'testnet':
                # Connect to testnet
                if self.exchange_id == 'binance':
                    self.exchange = ccxt.binance({
                        'apiKey': os.getenv('rKI3q7RZSOwaKmtEdpJr8gzW2ck32ZXVAUWhwajXU88pHlrm0cnKqyHWy2n2gbRJ'),
                        'secret': os.getenv('RfA74xF2DloTmJcHSVsIMnpk7YbHhfI4iV1SVv8NIR1pHf1rnCl8n14hfNSWXTiu'),
                        'enableRateLimit': True,
                        'options': {
                            'defaultType': 'future',
                            'adjustForTimeDifference': True,
                            'testnet': True
                        }
                    })
                else:
                    raise ValueError(f"Testnet for {self.exchange_id} not implemented")
            
            # Set leverage for futures
            if self.mode in ['testnet', 'live']:
                self.exchange.setLeverage(self.leverage, self.symbol)
                
            print(f"Successfully connected to {self.exchange_id}")
            
        except Exception as e:
            print(f"Error connecting to exchange: {e}")
            raise
    
    def update_market_price(self):
        """Get current market price"""
        if self.mode == 'paper':
            # Simulate a fluctuating price for paper trading
            self.initial_price = 30000 + np.random.uniform(-1000, 1000)  # Simulate price fluctuation
        else:
            ticker = self.exchange.fetch_ticker(self.symbol)
            self.initial_price = ticker['last']
        
        print(f"Current price: {self.initial_price}")
        return self.initial_price
    
    def generate_grid_prices(self):
        """Generate grid prices based on current price"""
        current_price = self.initial_price if self.initial_price else self.update_market_price()
        
        if self.grid_type == 'arithmetic':
            self.grid_prices = sorted([
                round(current_price + (self.grid_spacing * i), 2) 
                for i in range(-self.grid_levels // 2, self.grid_levels // 2 + 1)
            ])
        else:  # geometric
            self.grid_prices = sorted([
                round(current_price * (self.ratio ** i), 2) 
                for i in range(-self.grid_levels // 2, self.grid_levels // 2 + 1)
            ])
        
        print(f"Grid prices: {self.grid_prices}")
        return self.grid_prices
    
    def process_paper_trades(self, current_price):
        """Simulate processing paper trades"""
        # Check for active orders and simulate order completion
        for order_id, order in list(self.active_orders.items()):
            if order['status'] == 'open':
                if (order['side'] == 'BUY' and current_price <= order['price']) or \
                   (order['side'] == 'SELL' and current_price >= order['price']):
                    # Simulate filling the order
                    self.complete_order(order_id)
                    print(f"Paper order completed: {order['side']} {order['amount']} at {order['price']} {self.quote_currency}")
                
    def complete_order(self, order_id):
        """Complete a paper order and update the balance"""
        order = self.active_orders.pop(order_id)
        price = order['price']
        amount = order['amount']
        side = order['side']
        
        if side == 'BUY':
            # Simulate buying
            self.paper_balance -= price * amount
            self.paper_positions[order_id] = amount
        elif side == 'SELL':
            # Simulate selling
            self.paper_balance += price * amount
            del self.paper_positions[order_id]
        
        self.completed_trades += 1
        print(f"Order {order_id} completed: {side} {amount} at {price}. New balance: {self.paper_balance} USDT")
    
    def run_bot(self):
        """Main method to run the bot"""
        try:
            while True:
                # Simulate fetching the current market price
                current_price = self.update_market_price()
                
                # Process paper trades (if any)
                print(f"Processing trades at price {current_price}")
                self.process_paper_trades(current_price)
                
                # Simulate the grid strategy by placing orders (paper orders)
                for price in self.grid_prices:
                    # Check if buy or sell orders need to be placed
                    if current_price <= price and f"buy-{price}" not in self.active_orders:
                        self.place_order('BUY', price, self.order_size)
                    elif current_price >= price and f"sell-{price}" not in self.active_orders:
                        self.place_order('SELL', price, self.order_size)
                
                # Sleep for a while before checking again (simulating the real-time process)
                time.sleep(10)
                
        except KeyboardInterrupt:
            print("Bot stopped.")
            self.cancel_all_orders()
            print(f"Total completed trades: {self.completed_trades}")
            print(f"Total profit: {self.total_profit}")
    
    def place_order(self, side, price, size):
        """Place order based on mode"""
        effective_size = size * self.leverage
        order_id = None
        
        try:
            if self.mode == 'paper':
                # Simulate order placement
                order_id = f"paper-{int(time.time())}-{side}-{price}"
                self.active_orders[order_id] = {
                    'id': order_id,
                    'symbol': self.symbol,
                    'side': side,
                    'price': price,
                    'amount': effective_size,
                    'status': 'open',
                    'timestamp': int(time.time() * 1000)
                }
                print(f"Paper order placed: {side} {effective_size} {self.base_currency} at {price} {self.quote_currency}")
            else:
                # Place real order on testnet or live
                order_params = {
                    'symbol': self.symbol,
                    'type': 'LIMIT',
                    'side': side,
                    'amount': effective_size,
                    'price': price,
                    'timeInForce': 'GTC'  # Good Till Cancelled
                }
                
                order = self.exchange.create_order(
                    self.symbol,
                    'limit',
                    side.lower(),
                    effective_size,
                    price
                )
                order_id = order['id']
                self.active_orders[order_id] = order
                print(f"{self.mode.capitalize()} order placed: {side} {effective_size} {self.base_currency} at {price} {self.quote_currency}")
            
            return order_id
        except Exception as e:
            print(f"Error placing order: {e}")
            return None

if __name__ == "__main__":
    bot = GridTradingBot(mode='paper', symbol='BTC/USDT', exchange_id='binance')
    bot.run_bot()


Connected to paper trading (simulated).
Successfully connected to binance
Current price: 29187.963790919253
Grid prices: [1072.63, 2076.82, 4021.15, 7785.77, 15074.84, 29187.96, 56513.85, 109422.35, 211863.99, 410211.9, 794253.91]
Grid Trading Bot initialized in paper mode
Symbol: BTC/USDT, Exchange: binance
Initial price: 29187.963790919253
Grid type: geometric
Grid levels: 10
Leverage: 1x
Current price: 29513.315283316937
Processing trades at price 29513.315283316937
Paper order placed: SELL 0.001 BTC at 1072.63 USDT
Paper order placed: SELL 0.001 BTC at 2076.82 USDT
Paper order placed: SELL 0.001 BTC at 4021.15 USDT
Paper order placed: SELL 0.001 BTC at 7785.77 USDT
Paper order placed: SELL 0.001 BTC at 15074.84 USDT
Paper order placed: SELL 0.001 BTC at 29187.96 USDT
Paper order placed: BUY 0.001 BTC at 56513.85 USDT
Paper order placed: BUY 0.001 BTC at 109422.35 USDT
Paper order placed: BUY 0.001 BTC at 211863.99 USDT
Paper order placed: BUY 0.001 BTC at 410211.9 USDT
Paper order 

KeyError: 'paper-1740764814-SELL-1072.63'